In [1]:
#因为在远程通过XServer调用的Chrome要加下面一行
%env DISPLAY=211.139.119.100:0
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import json

#from bs4 import BeautifulSoup
#from IPython.display import Image
import time,random,base64

env: DISPLAY=211.139.119.100:0


In [2]:
chrome_options = webdriver.chrome.options.Options()
#chrome_options.add_argument("--headless")
d = DesiredCapabilities.CHROME
#这个不行
d['loggingPrefs'] = { 'performance':'ALL', 'driver':'ALL', 'browser':'ALL' }
#这个才可以在Chrome 78中使用
d = {
    'browserName': 'chrome',
    'loggingPrefs': {
        'browser': 'ALL',
        'driver': 'ALL',
        'performance': 'ALL',
    },
    'goog:chromeOptions': {
        'perfLoggingPrefs': {
            'enableNetwork': True,
        },
        'w3c': False, 
    },
}
browser = webdriver.Chrome(desired_capabilities=d)

In [3]:
browser.get('https://www.ximalaya.com/')
#在Chrome上进行登陆，并转至需要下载的节目的页面

In [4]:
print(browser.current_url)
#print(browser.page_source)

https://www.ximalaya.com/youshengshu/22630007/


In [5]:
def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

In [29]:
def start_play(num):
    #确定音频标题
    m4a_title = browser.find_element_by_xpath('//*[@id="anchor_sound_list"]/div[2]/ul/li[{}]/div[2]/a/span'.format(num)).text
    #print(m4a_title)
    #查找元素
    ximalaya_player=browser.find_element_by_xpath('//*[@id="anchor_sound_list"]/div[2]/ul/li[{}]/div[1]/div/div[1]/span'.format(num))
    time.sleep(1)
    #将元素可见
    browser.execute_script("arguments[0].scrollIntoView();",ximalaya_player)
    time.sleep(1)
    #向下滚动200个像素点
    browser.execute_script("window.scrollBy(0,-200)")
    time.sleep(1)
    if num == 1:
        ActionChains(browser).move_to_element(ximalaya_player).perform()
        time.sleep(2)
    #点击播放
    ximalaya_player=browser.find_element_by_xpath('//*[@id="anchor_sound_list"]/div[2]/ul/li[{}]/div[1]/div'.format(num))
    #在具体处理之前先清理log，不然会取到前一个文件的Log
    browser_log = browser.get_log('performance')
    #print(browser_log)
    #print("test")
    ximalaya_player.click()
    time.sleep(2)
    #分析log查找具体内容，Chrome的Network中的内容
    browser_log = browser.get_log('performance') 
    events = [process_browser_log_entry(entry) for entry in browser_log]

    ana_events = [event for event in events if 'Network.responseReceived' in event['method']]
    test_params = [test_param['params'] for test_param in ana_events]
    test1_params = [test1_param for test1_param in test_params if 'response' in test1_param.keys()]
    test2_params = [test2_param['response']['url'] for test2_param in test1_params if 'Media' in test2_param['type']]
    m4a_url = str(list(set(test2_params))[0])
    #print(m4a_url)
    #wget比网页下载要好，wget可以获取文件的真实创建时间
    print('wget "{}" -O {}.m4a'.format(m4a_url, m4a_title))
    print('sleep 5')
    time.sleep(1)
    #暂停播放
    ximalaya_player=browser.find_element_by_xpath('//*[@id="anchor_sound_list"]/div[2]/ul/li[{}]/div[1]/div'.format(num))
    ximalaya_player.click()
    time.sleep(1)

In [38]:
#一页30个音频
for m4a_num in range(1,31):
    start_play(m4a_num)

wget "https://vod.xmcdn.com/download/1.0.0/group3/M05/2D/77/wKgMbF34S-yAG_iMAH7pXMXi5vM936.m4a?sign=66c9c6fd23785d79da568be532fec88c&buy_key=617574686f72697a6564&token=4791&timestamp=1577413204&duration=1027" -O 平凡的世界210.m4a
sleep 5
wget "https://vod.xmcdn.com/download/1.0.0/group3/M02/31/D1/wKgMbF35nVfTwWtzAIUhZQ6ukig832.m4a?sign=549a91b1b57c49f97a2dfe8ef12abd74&buy_key=617574686f72697a6564&token=4249&timestamp=1577413211&duration=1065" -O 平凡的世界211.m4a
sleep 5
wget "http://audiopay.cos.xmcdn.com/download/1.0.0/group3/M04/31/D2/wKgMbF35nV2jwXUjAHkOFz2eqW4025.m4a?sign=27a3f68e6c935bbb9e9b23257a8f1920&buy_key=617574686f72697a6564&token=1357&timestamp=1577413219&duration=979" -O 平凡的世界212.m4a
sleep 5
wget "https://vod.xmcdn.com/download/1.0.0/group3/M00/35/E2/wKgMdl367t3CF13VAIHWQ9VVqxg830.m4a?sign=6bc27ad8ef152adbee6954da126d8ed2&buy_key=617574686f72697a6564&token=4690&timestamp=1577413226&duration=1050" -O 平凡的世界213.m4a
sleep 5
wget "https://vod.xmcdn.com/download/1.0.0/group3/M00/35/E2/w

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="anchor_sound_list"]/div[2]/ul/li[15]/div[2]/a/span"}
  (Session info: chrome=78.0.3904.108)
  (Driver info: chromedriver=78.0.3904.70 (edb9c9f3de0247fd912a77b7f6cae7447f6d3ad5-refs/branch-heads/3904@{#800}),platform=Linux 4.15.0-72-generic x86_64)


In [ ]:
#文件下载的函数
import requests
def download_file_as(Download_address, target_file)
#下载地址
#把下载地址发送给requests模块
down_file=requests.get(Download_address)
#下载文件
with open(target_file,"wb") as code:
     code.write(down_file.content)

In [24]:
start_play(2)

['https://vod.xmcdn.com/download/1.0.0/group2/M0B/EC/59/wKgLdF1vSk2zQmQuAHUqfSuz300781.m4a?sign=cf5f3c4d43bd0f46c953e0bc9ae51dbc&buy_key=617574686f72697a6564&token=4829&timestamp=1577410136&duration=948']
wget "https://vod.xmcdn.com/download/1.0.0/group2/M0B/EC/59/wKgLdF1vSk2zQmQuAHUqfSuz300781.m4a?sign=cf5f3c4d43bd0f46c953e0bc9ae51dbc&buy_key=617574686f72697a6564&token=4829&timestamp=1577410136&duration=948" -O 平凡的世界01（一口气听完《百年孤独》，爽！）.m4a
sleep 5
